# Understand correlation Red/blue wrt parameters

- Author Sylvie Dagoret-Campagne
- Afflilation : IJCLab/IN2P3/CNRS
- Organisation : LSST-DESC
- creation date : 2023-12-08
- last update : 2023-12-10

In [ ]:
import h5py
import pandas as pd
import numpy as np
import os
import re
import pickle 
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors
import matplotlib.cm as cmx
import collections
from collections import OrderedDict
import re
import matplotlib.gridspec as gridspec
import arviz as az

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['legend.fontsize']=  16

## imports

### fitter jaxopt

In [ ]:
from fors2tostellarpopsynthesis.fitters.fitter_jaxopt import (SSP_DATA,mean_spectrum,mean_mags,mean_sfr,ssp_spectrum_fromparam)

In [ ]:
from dsps.cosmology import DEFAULT_COSMOLOGY, age_at_z

In [ ]:
from fors2tostellarpopsynthesis.fitters.fitter_util import plot_params_kde,calc_ratio

### fors2starlightio

In [ ]:
from fors2tostellarpopsynthesis.fors2starlightio import  flux_norm

### parameters

In [ ]:
from fors2tostellarpopsynthesis.parameters import SSPParametersFit,paramslist_to_dict

## Configuration

### Constants

In [ ]:
Lyman_lines = [1220., 1030. ,973.,950., 938., 930.]
Balmer_lines = [6562.791,4861.351,4340.4721,4101.740,3970.072,3889.0641,3835.3971]
Paschen_lines = [8750., 12820., 10938.0,10050., 9546.2, 9229.7,9015.3, 8862.89,8750.46,8665.02]
Brackett_lines = [40522.79, 26258.71, 21661.178, 19440., 18179.21]
Pfund_lines = [ 74599.0, 46537.8, 37405.76 , 32969.8, 30400.]
all_Hydrogen_lines = [ Lyman_lines, Balmer_lines, Paschen_lines, Brackett_lines, Pfund_lines]
Color_lines = ["purple", "blue", "green", "red","grey"]
Balmer_thres = 3645.6
Lyman_thres = 911.267
Paschen_thres = 8200.
Brackett_thres = 14580.
Pfund_lines = 22800.
all_Hydrogen_thres = [Lyman_thres , Balmer_thres, Paschen_thres, Brackett_thres, Pfund_lines]

In [ ]:
wl0 = 3645.6

In [ ]:
D4000_red = [4050.,4250] 
D4000_blue = [3750.,3950.]
W_BALMER = [Balmer_thres, Balmer_lines[0]]
W_LYMAN = [Lyman_thres, Lyman_lines[0]]

In [ ]:
def plot_hydrogen_lines(ax):
    nth = len(all_Hydrogen_thres)
    for idx,group_lines in enumerate(all_Hydrogen_lines):
        color = Color_lines[idx]
        for wl_line in group_lines:
            ax.axvline(wl_line,color=color,lw=0.5)
        if idx< nth:
            ax.axvline(all_Hydrogen_thres[idx],color=color,linestyle=":")
    ax.axvspan(W_LYMAN[0],W_LYMAN[1],facecolor='green', alpha=0.5)
    ax.axvspan(W_BALMER[0],W_BALMER[1],facecolor='yellow', alpha=0.5)

### fitted params

In [ ]:
input_path = "fitssp_results"
input_file = "fitssp_results.h5"

In [ ]:
fullname_input = os.path.join(input_path ,input_file) 

### Functions

In [ ]:
def bluefraction(x,y,wlcut=wl0):

    indexes_blue = np.where(x<wlcut)[0]
    indexes_red =np.where(x>wlcut)[0]

    integ_blue = np.trapz(y[indexes_blue],x[indexes_blue])
    integ_red = np.trapz(y[indexes_red],x[indexes_red])

    fraction_blue = integ_blue /(integ_blue+integ_red)
    fraction_red= integ_red /(integ_blue+integ_red)

    return fraction_blue

## Read Dataframe 

In [ ]:
df = pd.read_hdf(fullname_input)

In [ ]:
df

In [ ]:
N = len(df)

In [ ]:
df

In [ ]:
# create colormap
#cm = plt.cm.bwr(np.linspace(0, 1, N))

## Decode fitted DSPS parameters and plot spectra

In [ ]:
names_fitparams = list(df.columns[8:])

In [ ]:
all_x = []
all_y_nodust = []
all_y_dust = []
the_ymax= 1e-12
all_fractions_blue = []
all_BL = []
all_D4000 = []



# loop on fitted spectra
for idx in range(N):

    # SSP
    row = df.iloc[idx]
    z_obs = row["zobs"]
    specname = row["fors2name"]
    list_params = row[names_fitparams].values
    dict_params = paramslist_to_dict(list_params,names_fitparams)
    x,y_nodust,y_dust = ssp_spectrum_fromparam(dict_params,z_obs)

    norm_y_nodust = flux_norm(x,y_nodust,wlcenter=wl0)
    norm_y_dust = flux_norm(x,y_dust,wlcenter=wl0)

    y_nodust /=  norm_y_nodust
    y_dust /=  norm_y_dust
    
    fract_blue = bluefraction(x,y_nodust)
    D4000 = calc_ratio(x,y_nodust)
    DBL = calc_ratio(x,y_nodust,W_LYMAN ,W_BALMER )
    
    
    all_x.append(x)
    all_y_nodust.append(y_nodust) 
    all_y_dust.append(y_dust) 
    all_fractions_blue.append(fract_blue)
    all_BL.append(DBL)
    all_D4000.append(D4000)

    ymax = y_nodust.max()
    the_ymax = max(the_ymax,ymax)
    

ylim_max = the_ymax*2.
ylim_min = ylim_max/1e7

all_fractions_blue = np.array(all_fractions_blue)
all_BL = np.array(all_BL)
all_D4000 = np.array(all_D4000)

In [ ]:
fig = plt.figure(figsize=(8,6))
axs = fig.subplots(2,2)

for idx, ax in enumerate(np.ravel(axs)):
    if idx==0:
        ax.hist(all_D4000,bins=50,density=True,facecolor="r",label="DSPS")
        ax.set_xlabel("D4000")
        ax.legend()
    elif idx==1:
        ax.hist(np.log10(all_BL),bins=50,density=True,facecolor="r")
        ax.set_xlabel("Balmer/Lyman")
        ax.set_yscale('log')
    elif idx==2:
        ax.scatter(all_D4000,np.log10(all_BL),marker='o',alpha=0.3,facecolor="r")
        ax.set_xlabel("D4000")
        ax.set_ylabel("Balmer/Lyman")
    elif idx==3:
        ax.hist(np.log10(all_fractions_blue),density=True,facecolor="r",alpha=0.8)
        ax.set_xlabel("Blue fraction")
#ax.set_title("D4000")
#ax.set_xlabel("D4000")
plt.tight_layout()

In [ ]:
bwr_map = plt.get_cmap('bwr')
reversed_map = bwr_map.reversed() 
#cNorm = colors.Normalize(vmin=np.log10(all_fractions).min(), vmax=np.log10(all_fractions).max())
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=reversed_map)
#all_colors = scalarMap.to_rgba(np.log10(all_fractions), alpha=1)
cNorm = colors.Normalize(vmin=all_D4000.min(), vmax=all_D4000.max())
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=bwr_map)
all_colors = scalarMap.to_rgba(all_D4000, alpha=1)

In [ ]:
_, ax1 = plt.subplots(1, 1,figsize=(10,5))


__= ax1.set_yscale('log') 
__= ax1.set_xscale('log') 

for idx in range(N):
    ax1.plot(all_x[idx],all_y_nodust[idx],'-',color=all_colors[idx])


__= ax1.set_xlim(100.,1e6)
__= ax1.set_ylim(ylim_min ,ylim_max )

ax1.set_xlabel("$\lambda (\\AA)$")
ax1.set_ylabel("$L_\\nu(\lambda)$ relative flux")
ax1.set_title("$f_\\nu(\lambda)$ Fitted spectra")
ax1.grid()
#ax1.axvline(wl0,color="k")
plot_hydrogen_lines(ax1)

plt.tight_layout()

In [ ]:
_, ax1 = plt.subplots(1, 1,figsize=(10,5))
__= ax1.set_yscale('log') 
__= ax1.set_xscale('log') 

for idx in range(N):
    ax1.plot(all_x[idx],all_y_nodust[idx]/(all_x[idx]/wl0)**2,color=all_colors[idx])
__= ax1.set_xlim(100.,1e5)
#__= ax1.set_ylim(1e-12 ,1e-4 )
__= ax1.set_ylim(1e-3 ,1e2 )

#ax1.axvline(wl0,color="k")
ax1.set_xlabel("$\lambda (\\AA)$")
ax1.set_ylabel("$L_\\lambda(\lambda)$ relative flux")
ax1.set_title("$f_\\lambda(\lambda)$ Fitted spectra")
plot_hydrogen_lines(ax1)
ax1.grid()


plt.tight_layout()

## Correlation

In [ ]:
names_fitparams

In [ ]:
for idx0,selected_paramname in enumerate(names_fitparams):

    all_param_values = []
    # loop on fitted spectra
    for idx1 in range(N):

        # SSP
        row = df.iloc[idx1]
        z_obs = row["zobs"]
        specname = row["fors2name"]
        param_value = row[selected_paramname]
        all_param_values.append(param_value )
     
    all_param_values = np.array(all_param_values)
    
    dict_data = {'D4000':all_D4000,selected_paramname:all_param_values}
    
    try:
        
        _, ax = plt.subplots(1, 1,figsize=(3,3))
        az.plot_pair(dict_data,var_names=['D4000', selected_paramname], kind='kde',ax=ax)
        plt.show()
    except Exception as inst:

        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)    
    
    
    